# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [17]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
  print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
  print("OpenAI API Key not set")

if anthropic_api_key:
  print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
  print("Anthropic API Key not set (and this is optional)")

MODEL_GPT = 'gpt-5'
MODEL_CLAUDE = 'claude-sonnet-4-6'

anthropic_url = "https://api.anthropic.com/v1/"

openai = OpenAI()
anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)

MODELS = {
  "gpt-5": openai,
  "claude-sonnet-4-6": anthropic
}

system_message = """
You are a helpful assistant that answers technical questions including code explanations. You should explain the concepts clearly with practical examples, in a way that any layman would understand.
"""

def chat_stream(model, question, history):
  history = [{"role": h["role"], "content": h["content"]} for h in history]
  messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": question}]

  response = MODELS[model].chat.completions.create(
    model=model,
    messages=messages,
    stream=True
  )

  partial = ""
  for chunk in response:
    delta = getattr(chunk.choices[0].delta, "content", None)
    if delta:
      partial += delta
      yield history + [{"role": "assistant", "content": partial}]

def put_message_in_chatbot(message, history):
  return "", history + [{"role":"user", "content":message}]

def transcribe_and_chat(audio_filepath, history):
  if not audio_filepath:
    return history
  with open(audio_filepath, "rb") as audio_file:
    transcript = openai.audio.transcriptions.create(
      model="whisper-1",
      file=audio_file
    )

  message = transcript.text
  return history + [{"role":"user", "content":message}]

with gr.Blocks() as ui:
  with gr.Row():
    model_selector = gr.Radio(label="Model", choices=list(MODELS.keys()), value=MODEL_GPT)
  with gr.Row():
    chatbot = gr.Chatbot(height=500, type="messages")
  with gr.Row():
    with gr.Column():
      question_input = gr.Textbox(label="Your Question", lines=2)
      submit_btn = gr.Button("Ask")
    with gr.Column():
      audio_input = gr.Audio(sources="microphone", type="filepath", label="Or speak your question, then click the stop button on the audio player and then click Ask with Audio. Click the close button to clear the audio input amd record another message.")
      submit_audio_btn = gr.Button("Ask with Audio")

  submit_btn.click(put_message_in_chatbot, inputs=[question_input, chatbot], outputs=[question_input, chatbot]).then(
    chat_stream, inputs=[model_selector, question_input, chatbot], outputs=[chatbot], api_name=None
  )

  submit_audio_btn.click(transcribe_and_chat, inputs=[audio_input, chatbot], outputs=[chatbot], api_name=None).then(
    chat_stream, inputs=[model_selector, question_input, chatbot], outputs=[chatbot], api_name=None
  )

ui.launch(inbrowser=True)

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.
